In [1]:
import pandas as pd
import numpy as np


In [2]:
dataset=pd.read_csv("preplacement.csv")

In [3]:
dataset.drop(columns="sl_no",inplace =True)

In [4]:

def quan_qual(dataset):
    quan=[]
    qual=[]
    for columnname in dataset:
        if dataset[columnname].dtype=="O":
             qual.append(columnname)
        else :
            quan.append(columnname)
    return quan,qual 

In [5]:
quan,qual=quan_qual(dataset)

In [6]:
quan

['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary']

In [7]:
def vif(dataset):
    import numpy as np
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    
    dataset=dataset[quan]
    values=dataset.values
    vif=[variance_inflation_factor(values,i)for i in range(values.shape[1])]
    vif_table=pd.DataFrame({"vif_values":vif,"variables":dataset.columns})
    return vif_table

In [8]:
vif(dataset)

,vif_values,variables
0,76.496603,ssc_p
1,58.972066,hsc_p
2,113.299298,degree_p
3,32.679659,etest_p
4,114.025972,mba_p
5,3.931112,salary


# 1. to drop the columns with high multicollinearity repeat until all vif is less 

# than 10

In [9]:
dataset.columns

Index(['gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p',
       'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p', 'status',
       'salary'],
      dtype='object')

In [10]:
dataset_drop=dataset.drop(columns=["degree_p","ssc_p","mba_p"])

In [11]:
dataset_drop

,gender,ssc_b,hsc_p,hsc_b,hsc_s,degree_t,workex,etest_p,specialisation,status,salary
0,M,Others,91.00,Others,Commerce,Sci&Tech,No,55.0,Mkt&HR,Placed,270000.0
1,M,Central,78.33,Others,Science,Sci&Tech,Yes,86.5,Mkt&Fin,Placed,200000.0
2,M,Central,68.00,Central,Arts,Comm&Mgmt,No,75.0,Mkt&Fin,Placed,250000.0
3,M,Central,52.00,Central,Science,Sci&Tech,No,66.0,Mkt&HR,Not Placed,0.0
4,M,Central,73.60,Central,Commerce,Comm&Mgmt,No,96.8,Mkt&Fin,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...
210,M,Others,82.00,Others,Commerce,Comm&Mgmt,No,91.0,Mkt&Fin,Placed,400000.0
211,M,Others,60.00,Others,Science,Sci&Tech,No,74.0,Mkt&Fin,Placed,275000.0
212,M,Others,67.00,Others,Commerce,Comm&Mgmt,Yes,59.0,Mkt&Fin,Placed,295000.0
213,F,Others,66.00,Others,Commerce,Comm&Mgmt,No,70.0,Mkt&HR,Placed,204000.0


In [12]:
quan,qual=quan_qual(dataset=dataset_drop) # always after calling quan name the variable quan qual cos the list is inside the function
# and u need to define it outside the function

In [13]:
quan

['hsc_p', 'etest_p', 'salary']

In [14]:
vif(dataset_drop)

,vif_values,variables
0,25.937543,hsc_p
1,22.845092,etest_p
2,3.132731,salary


# 2. principal component analysis



In [15]:
# first we hv to standardise our data using standard scalar
from sklearn.preprocessing import StandardScaler

In [16]:
x_tab=dataset[['gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p',
       'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p', 'status']]

In [17]:
x_tab=pd.get_dummies(x_tab,dtype=int,drop_first=True)

In [18]:
sc=StandardScaler()

In [19]:
sc.fit_transform(x_tab)

array([[-0.02808697,  2.2688123 , -1.14010225, ..., -0.72444647,
         1.12390297,  0.67283185],
       [ 1.11336869,  1.10344799,  1.51326671, ...,  1.38036423,
        -0.88975652,  0.67283185],
       [-0.21323793,  0.15331275, -0.32284282, ..., -0.72444647,
        -0.88975652,  0.67283185],
       ...,
       [-0.02808697,  0.06133451,  0.90304633, ...,  1.38036423,
        -0.88975652,  0.67283185],
       [ 0.61994138, -0.03064373, -1.14010225, ..., -0.72444647,
         1.12390297,  0.67283185],
       [-0.49096436, -0.76646966, -1.82115177, ..., -0.72444647,
         1.12390297, -1.48625544]])

In [20]:
from sklearn.decomposition import PCA


In [46]:
pca = PCA(n_components=2)  # Choose number of principal components, we can hv upto n number where n=no.of independents in our data.
X_pca = pca.fit_transform(x_tab) #here pca will analzse the standardised data and take 2 component or axes for that data which shows
# maximum variance

In [30]:
pc1,pc2=pca.explained_variance_ratio_ #this shows that axes 1 shows 50 percent varianc e with our data
#axes 2 shows 26 percent variance and the other 12 will show only approx 14 percent variance stating they r hightly multicollinear.

In [31]:
pc1

0.5020590684886636

In [32]:
pc2

0.26161523735460934

In [26]:
sum(pca.explained_variance_ratio_) 

In [33]:
# once we get he pc1,pc2 values we see the datapoints with these variances .pc1 split among 215 datapoints and same with pc2.

In [37]:
pca_table=pd.DataFrame(X_pca,columns=["pc1","pc2"])

In [45]:
pca_table

,pc1,pc2
0,-1.257138,-21.434164
1,24.851332,-0.890274
2,-0.055865,3.431124
3,-21.300829,8.677422
4,28.986670,8.446565
...,...,...
210,31.581623,-0.115980
211,-6.754871,7.463212
212,-4.593869,-12.550031
213,-0.727800,-2.334848
